In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [2]:
!pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1746434530, 'localtime': '2025-05-05 01:42'}, 'current': {'last_updated_epoch': 1746433800, 'last_updated': '2025-05-05 01:30', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 339, 'wind_dir': 'NNW', 'pressure_mb': 1009.0, 'pressure_in': 29.79, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 80, 'cloud': 0, 'feelslike_c': 12.7, 'feelslike_f': 54.8, 'windchill_c': 12.7, 'windchill_f': 54.9, 'heatindex_c': 12.6, 'heatindex_f': 54.6, 'dewpoint_c': 10.1, 'dewpoint_f': 50.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 12.2, 'gust_kph': 19.7}}", 'score': 0.826

In [5]:
!pip install -qU "langchain[google-genai]"

In [6]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

Enter API key for Google Gemini:  ········


In [7]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I help you today?'

In [8]:
model_with_tools = model.bind_tools(tools)

In [9]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi there! How can I help you today?
ToolCalls: []


In [10]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'c010fc74-489f-4b56-9036-06ee6e8ac2ae', 'type': 'tool_call'}]


In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [12]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='95e6872e-77bc-482e-afed-9a94d35744f2'),
 AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f1f32018-0729-4a32-ab7b-1212397a2b21-0', usage_metadata={'input_tokens': 51, 'output_tokens': 11, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}})]

In [13]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='f56e967e-b51c-4d69-948a-de49ba4f3d3a'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--81f7ff31-ee8f-4a66-a7a5-300b120db38a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': '5a5b3327-56ee-4284-b046-af5f5b05b9f2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 13, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"title": "San Francisco weather in May 2025 | Weather25.com", "url": "https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=May", "content": "Location 

In [14]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (32e25e38-b78c-47f7-83a3-da91e4d53f6d)
 Call ID: 32e25e38-b78c-47f7-83a3-da91e4d53f6d
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "San Francisco weather in May 2025 | Weather25.com", "url": "https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=May", "content": "Location was added to My Locations\nLocation was removed from My Locations\nApril\nJune\nSan Francisco weather in May 2025\nThe temperatures in San Francisco in May are usually low and can range between 50°F and 68°F.\nThe wather in San Francisco in May can vary between cold and nice weather days. Expect a few rainy days but usually not more than 3

In [15]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

I|'m sorry, I don't have the current weather conditions in San Francisco.| However, I can tell you that the expected temperatures in San Francisco in May are| usually between 50°F and 68°F.|

In [16]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in San| Francisco in May 2025 is expected to have daytime maximum temperatures of| 18°C, falling to 10°C at night. There| are expected to be 11 hours of sunshine per day on average.|

In [17]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [18]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--1959fc33-f6f7-4c2a-9116-e1cb78ad27e8-0', usage_metadata={'input_tokens': 53, 'output_tokens': 11, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})]}}
----


In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--c4627a6d-d2a1-4ce8-a348-57a54b7db6cf-0', usage_metadata={'input_tokens': 67, 'output_tokens': 6, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}})]}}
----


In [21]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I do not have access to personal information, and thus I don't know your name.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a5e19987-3fa7-400a-ab3a-816179ce207b-0', usage_metadata={'input_tokens': 53, 'output_tokens': 20, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}})]}}
----


In [23]:
config = {"configurable": {"thread_id": "xyz242"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hey, what's up?")]}, config
):
    print(chunk)

{'agent': {'messages': [AIMessage(content="I'm doing well, thanks for asking! How can I help you with anything?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--360a495d-032b-46cb-b20c-cf62f85d147c-0', usage_metadata={'input_tokens': 76, 'output_tokens': 19, 'total_tokens': 95, 'input_token_details': {'cache_read': 0}})]}}
